In [14]:
from numpy.polynomial import Polynomial


In [15]:

def karatsuba(a: Polynomial, b: Polynomial):
    """
    Multiply two polynomials using the Karatsuba method.

    Karatsuba works by splitting each polynomial into two halves:
        a(x) = a0(x) + y * a1(x)
        b(x) = b0(x) + y * b1(x)

    and then computing:
        c0 = a0 * b0
        c2 = a1 * b1
        c1 = (a0 + a1)*(b0 + b1) - c0 - c2

    Final result:
        c(x) = c0 + c1 * y + c2 * y^2
    """

    # base case (if either polynomial has degree 0, use normal multiplication)
    if len(a.coef.tolist()) == 1 or len(b.coef.tolist()) == 1:
        return a * b


    # step 1: ensure each polynomial has an even number of coefficients. karatsuba splits the coefficient list exactly in half. If it's odd, we append a zero so the split is even.
    a = check_order(a)
    b = check_order(b)

    # step 2: Pad both polynomials to the same length and compute y (refer to ppt slide 173). This ensures a and b have equal size, enabling clean splitting.
    a, b, y = check_length(a, b)

    # step 3: split each polynomial in half
    a_0, a_1 = split_polynomial_half(a)
    b_0, b_1 = split_polynomial_half(b)

    # step 4: recursive Karatsuba computation

    # c0 = a0 * b0
    c_0 = karatsuba(a_0, b_0)

    # c1 = a1 * b1
    c_2 = karatsuba(a_1, b_1)

    # c1 = (a0 + a1)(b0 + b1) - c0 - c2
    c_1 = karatsuba(a_0 + a_1, b_0 + b_1) - c_0 - c_2

    # final result
    return c_0 + (c_1 * y) + (c_2 * y**2)


In [16]:
def check_order(poly: Polynomial):
    """
    Ensures the polynomial has an even number of coefficients.
    Karatsuba splitting only works cleanly when the length is even.
    """
    order = len(poly.coef) - 1

    # If the number of coefficients is odd (even order), append a zero
    if (order % 2 == 0):
        new_coefs = poly.coef.tolist() + [0] 
        poly = Polynomial(new_coefs)
        
    return poly

In [17]:
def check_length(a: Polynomial, b: Polynomial):
    """
    Pads both polynomials to have equal length.
    Returns also y = x^m, where m is half the padded length.
    """

    a_coef = a.coef.tolist()
    b_coef = b.coef.tolist()

    max_deg = max(len(a_coef), len(b_coef))

    # Pad polynomial a to longest length
    if len(a_coef) < max_deg:
        a_coef = a_coef + [0] * (max_deg - len(a_coef))

    # Pad polynomial b to longest length
    if len(b_coef) < max_deg:
        b_coef = b_coef + [0] * (max_deg - len(b_coef))
    
    # Compute m = half of the padded length
    m = max_deg // 2

    # Create polynomial y = x^m (used for shifting)
    y = Polynomial([0] * m + [1])  # polynomial x^m


    return Polynomial(a_coef), Polynomial(b_coef), y


In [18]:
def split_polynomial_half(poly: Polynomial):
    """
    Splits a polynomial into two halves
    """

    coef = poly.coef.tolist()
    index = int(len(coef) / 2)
    left_half_coef = coef[: index]
    right_half_coef = coef[index:]

    left_half_poly = Polynomial(left_half_coef)
    right_half_poly = Polynomial(right_half_coef)
    
    return left_half_poly, right_half_poly


In [19]:
a = Polynomial([1, 3, 7, 9, 5, 7])
b = Polynomial([9, 1, 2, 3, 4, 2])

print(karatsuba(a, b))

9.0 + 28.0 x + 68.0 x**2 + 97.0 x**3 + 81.0 x**4 + 121.0 x**5 +
78.0 x**6 + 79.0 x**7 + 59.0 x**8 + 38.0 x**9 + 14.0 x**10
